<a href="https://colab.research.google.com/github/Neerajs004/Plant-Chatbot/blob/main/Plant_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import and Folder Creation
import os
#create project folder
base = "/content/PlantDocBot"
os.makedirs(os.path.join(base,"data","plantvillage"),exist_ok=True)
os.makedirs(os.path.join(base,"data","plantdoc"),exist_ok=True)
os.makedirs(os.path.join(base,"data","text_corpus"),exist_ok=True)

print("Folders created under",base)

Folders created under /content/PlantDocBot


In [ ]:
#Download Dataset via git clone
!git clone https://github.com/spMohanty/plantvillage-Dataset.git "{base}/data/plantvillage"
!git clone https://github.com/pratikkayal/PlantDoc-Dataset.git "{base}/data/plantdoc"

Cloning into '/content/PlantDocBot/data/plantvillage'...
remote: Enumerating objects: 163235, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 163235 (delta 2), reused 1 (delta 0), pack-reused 163229 (from 1)
Receiving objects: 100% (163235/163235), 2.00 GiB | 34.08 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Updating files: 100% (182401/182401), done.
Cloning into '/content/PlantDocBot/data/plantdoc'...
remote: Enumerating objects: 2670, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 2670 (delta 22), reused 22 (delta 22), pack-reused 2635 (from 1)
Receiving objects: 100% (2670/2670), 932.92 MiB | 42.01 MiB/s, done.
Resolving deltas: 100% (24/24), done.
Updating files: 100% (2581/2581), done.


In [ ]:
# Verify Dataset directories and list top-level content
for sub in ["plantvillage","plantdoc"]:
  path = os.path.join(base,"data",sub)
  print("\nContents of", sub, ":")
  print(os.listdir(path)[:20])


Contents of plantvillage :
['leaf_grouping', 'create_data_distribution.py', 'generate_data_segmented-80-20.sh', 'generate_data_grayscale-80-20.sh', 'slurm-476487.out', 'generate_data_segmented-60-40.sh', 'slurm-476488.out', 'generate_data_segmented-20-80.sh', 'generate_mapstring.py', 'slurm-476492.out', 'create_db.py', 'generate_data_grayscale-40-60.sh', 'slurm-476484.out', 'slurm-476482.out', 'slurm-476486.out', 'slurm-476481.out', 'generate_data_grayscale-50-50.sh', '.git', 'raw', 'slurm-476485.out']

Contents of plantdoc :
['LICENSE.txt', 'PlantDoc_Examples.png', '.git', 'README.md', 'train', 'test']


In [ ]:
#Search for image directories inside PlantVillage--
import os
pv_base = os.path.join(base,"data", "plantvillage")
img_exts = ('.jpg','.jpeg','.bmp')

found_dirs = []
for root_dir, dirs, files in os.walk(pv_base):
  count=sum(1 for f in files if f.lower().endswith(img_exts))
  if count>0:
    found_dirs.append((root_dir,count))
if not found_dirs:
    print("No Image files found inside PlantVillage folder .")
else:
    print("Found image directories. Sample list(first 10):")
    for d,c in found_dirs[:10]:
      print(" ",d, "-",c,"images")
    #choose first as img_root
    img_root=found_dirs[0][0]
    print("\nUsing image root:",img_root)

Found image directories. Sample list(first 10):
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Potato___healthy - 152 images
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Corn_(maize)___Common_rust_ - 1192 images
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Grape___healthy - 423 images
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Tomato___Early_blight - 1000 images
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Tomato___Tomato_mosaic_virus - 373 images
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Raspberry___healthy - 371 images
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Pepper,_bell___healthy - 1477 images
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Strawberry___Leaf_scorch - 1109 images
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Tomato___Bacterial_spot - 2127 images
  /content/PlantDocBot/data/plantvillage/raw/grayscale/Tomato___Leaf_Mold - 952 images

Using image root: /content/PlantDocBot/data

In [5]:
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
import numpy as np

#use img_root from previous cell
if'img_root' in globals():
  sample_file = None
  for root_dir, dirs, files in os.walk(img_root):
    img_files = [f for f in files if f.lower().endswith(img_exts)]
    if img_files:
      sample_file =os.path.join(root_dir, random.choice(img_files))
      break
      if img_file:
        print("Displaying color image:",sample_file)
        img = Image.open(sample_file)

        #Check mode
        print("Original image mode:",img.mode)

        #Convert to true RGB if not already
        if img.mode != 'RGB':
          img = img.convert('RGB')
        #Use Numpy+matplotlib to ensure correct color display
        plt.figure(figsize=(6,6))
        plt.imshow(np.asarray(img))
        plt.axis('off')
        plt.show()
    else:
          print("No images found under img_root.")
  else:
          print("img_root not defined-previous detection failed.")

In [9]:
#Build CSV Mapping image path <------
import pandas as pd
records = []
if 'img_root' in globals():
  for root_dir, dirs, files in os.walk(img_root):
    for f in files:
      if f.lower().endswith(img_exts):
        path = os.path.join(root_dir,f)
        #infer label:directory name relative to img_root
        rel = os.path.relpath(path,img_root)
        label = rel.split(os.sep)[0]  #first folder after img_root
        records.append({"image_path":path,"label":label})
df = pd.DataFrame(records)
print("Total images found:",len(df))
print("Sample rows:")
print(df.head())
out_csv=os.path.join(base,"data","image_data.csv")
df.to_csv(out_csv,index=False)
print("Saved mapping to",out_csv)

Total images found: 0
Sample rows:
Empty DataFrame
Columns: []
Index: []


NameError: name 'base' is not defined